

<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/ISOTIPO-Color.jpg" title="Title text" width="20%" />

<hr style="height:2px;border:none"/>
<h1 align='center'> INF-479 Reconocimiento de patrones en minería de datos </h1>

<H2 align='center'> Tarea 3 - Sistemas Recomendadores </H2>
<H3 align='center'> Nombre estudiante </H3>
<H3 align='center'> Rol </H3>
<hr style="height:2px;border:none"/>

# Instrucciones

1. La tarea debe ser desarrollada en parejas.  
2. Los medios de consultas serán Aula y el canal de Discord oficial del curso. 
3. La tarea debe ser realizada en Jupyter Notebook (Python3) utilizando este archivo como base. 
4. Se evaluará la correcta implementación de los algoritmos de filtrado colaborativo aplicados a los datasets escogido por cada grupo, como también la comprensión de los fundamentos teóricos, ventajas y desventajas de cada técnica. 
5. El archivo de entrega debe denominarse T3_Apellido1_Apellido2.ipynb. De no respetarse este formato existirá un descuento de 30 puntos. 
6. La fecha de entrega es el lunes 2 de agosto a las 23:55 hrs, posterior a esto se descontará 1 punto por cada minuto de atraso.  

In [4]:
#Librerías 
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors


## Datasets Tarea

Para la tarea tendrán a su disposición los siguientes 3 datasets, <b> de las cuales tendrán que escoger 2 para realizar la implementación de su tarea. </b>

   ### 1. MovieLens: 
   Es uno de los datasets más populares de recomendación y recopila las preferencias de usuarios de internet con respecto a películas que son evaluadas de 0 a 5 estrellas. Este dataset ha sido utilizado en diversos estudios de investigación en áreas como la recomendación personalizada y la psicología social.  
* <b>Archivos: </b>
    - ML_ratings.csv: contiene los ratings dados por usuarios a películas. Se compone por las columnas user_id, movie_id y rating. 
    - movies.csv: contiene la metadata sobre las películas. Incluye las columnas movie_id, title y genres (donde se encuentran los géneros de la película separados por "|"). 
 
* <b>Cantidad de ratings</b>: 100836
* <b>Cantidad de usuarios</b>: 610
* <b>Cantidad de películas</b>: 9724
* <b>Calificación</b> 0 a 5 (valores enteros) 

Referencias: [Movielens Dataset](https://grouplens.org/datasets/movielens/) 
   

In [6]:
df = pd.read_csv("ML_ratings.csv")
df.head()

,user_id,movie_id,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


### 2. Book-Crossing: 
Es un datasets de ratings de libros recolectado por Cai-Nicolas Ziegler desde la <i>Book-Crossing community</i>. El dataset original contiene 1,149,780 ratings generados por 278,858 usuarios a 271,379 libros. 

* <b>Archivos</b>: 
    - BX_ratings.csv: contiene los ratings dados por usuarios a diferentes libros. Se compone de las columnas user_id, ISBN (identificador para los libros) y rating.
    - books.csv: contiene los títulos de libros asociados a los códigos ISBN (utilizados como item_id). Se compone de las columnas ISBN y title. 

* <b>Cantidad de ratings</b>: 359263
* <b>Cantidad de usuarios</b>: 10775
* <b>Cantidad de libros</b>: 10773
* <b>Calificación</b>: 0 a 10 (valores enteros)

Referencias: [Book-Crossing Dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/)

In [11]:
df = pd.read_csv("BX_ratings.csv")
df.head()

,user_id,ISBN,rating
0,276762,034544003X,0
1,276762,0380711524,5
2,276762,0451167317,0
3,276762,3404611306,0
4,276798,3442437407,0


### 3. Jester: Es un dataset desarrollado por Ken Goldberg y su equipo en la Universidad de Berkeley que contiene alrededor de 6 millones de ratings con respecto a 150 chistes cortos. 
* <b>Archivos</b>:
    - JT_ratings.csv: contiene los ratings dados por usuarios a diferentes chistes cortos. Se compone por las columnas joke_id, user_id y rating. 
    - jokes.csv: contiene 

* <b>Cantidad de ratings</b>: 199900
* <b>Cantidad de usuarios</b>: 1999
* <b>Cantidad de chistes</b>: 100 
* <b>Calificación</b>: -10 a 10 (valores reales)
    
Referencias: [Jester Dataset](http://eigentaste.berkeley.edu/dataset/)

In [8]:
df = pd.read_csv("JT_ratings.csv")
df.head()

,joke_id,user_id,rating
0,0,1,5.10
1,1,1,4.90
2,2,1,1.75
3,3,1,-4.17
4,4,1,5.15


## Instrucciones

<img src="https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs11227-020-03266-2/MediaObjects/11227_2020_3266_Fig1_HTML.png" title="Title text" width="60%" />
<center> <i> Figura 1. Collaborative Filtering. </i> </center>

## 1. User based Collaborative Filtering (40 puntos) 

La primera parte de la tarea constará de las siguientes secciones: 
1. Implementación de un sistema de recomendación de filtro colaborativo basado en usuarios utilizando los k vecinos más cercanos. Para esta parte, se debe utilizar <b> similaridad coseno </b> como medida de similaridad entre el usuario objetivo y el resto de usuarios. Además, el parámetro k debe ser escogido por ustedes. 

2. Se ingresa nuevo usuario al sistema, se le pide que califique 10 productos (a elección) y a partir de eso se le realiza la recomendación de 5 productos que no ha calificado. Es importante que en su procedimiento se muestren las ids de los k vecinos más cercanos.  

3. Concluir y responder las siguientes preguntas: 

   >a. ¿Cuáles fueron las 5 recomendaciones obtenidas? Analice y concluya respecto a estos resultados.  
    
   >b. ¿Qué cantidad de vecinos cercanos (k) se escogió para la recomendación? ¿En qué influye la elección de este parámetro? ¿Qué sucede a medida que aumenta este parámetro?
    
   >c. ¿Cuál era el porcentaje de <i>sparsity</i> de la matriz usuarios-items? ¿Cuáles son las desventajas de este enfoque? 


<i>Importante: Esto debe ser realizado para ambos datasets escogidos</i>


## 2. Item based Collaborative Filtering (40 puntos) 

La segunda parte de la tarea constará de las siguientes secciones: 
1. Implementación de un sistema de recomendación de filtro colaborativo basado en ítems utilizando los k ítems más cercanos. Para esta parte, se debe utilizar <b> similaridad coseno </b> como medida de similaridad entre los ítems. Además, el parámetro k debe ser escogido por ustedes. 

2. Se ingresa nuevo usuario al sistema, se le pide que califique 10 productos (a elección) y a partir de eso se le realiza la recomendación de 5 productos que no ha calificado. Para poder llevar a cabo la recomendación recordar los siguientes pasos:
    >a. Se debe generar la matriz de similaridad entre productos basada en la similaridad coseno.
    
    >b. Para realizar la recomendación, se debe predecir el rating de todos los productos que el usuario nuevo del sistema no haya calificado aún. Para esto, se deben obtener los k ítems más cercanos al ítem a predecir y se debe predecir su rating en base a la <i> weighted sum </i> de los ratings de dichos k ítems. 
    
    >c. Una vez predecidos los ratings para todos los productos sin calificación, se deben recomendar al usuario los 5 productos con mayor rating predecido. 

3. Concluir y responder las siguientes preguntas: 

   >a. ¿Cuáles fueron las 5 recomendaciones obtenidas? Analice y concluya respecto a estos resultados.  
    
   >b. ¿Qué cantidad de vecinos cercanos (k) se escogió para la recomendación? ¿En qué influye la elección de este parámetro? ¿Qué sucede a medida que aumento este parámetro?
    
   >c. ¿Cuáles son las ventajas de este enfoque respecto al anterior? 


<i>Importante: Esto debe ser realizado para ambos datasets escogidos</i>

## 3. Conclusiones finales (20 puntos)

1. Analice los resultados obtenidos y contraste ambos enfoques (ubcf y ibcf). 
2. Analice ventajas y desventajas de ambos enfoques. 

### Recomendación K-neighbours 

Para calcular la búsqueda de los k vecinos más cercanos en su implementación, pueden hacer uso de la función de sklearn que se encuentra importada en el Notebook. Sin embargo, se sugiere que utilicen la librería [MRPT](https://github.com/vioshyvo/mrpt) debido a que se posiciona como una de las librerías más rápidas para la búsqueda de vecinos cercanos. 